In [2]:
import math
import ROOT
ObservableList = []

class Observable(object):
    def __init__(self, name, binning = (20, -1000, 1000), script = None, style = 'single', do = ['hist'], only = None, title = '{self.name}', dtype = float, default = -1, need_truth = False):
            """Observable for filling histograms or trees
            
            Parameters
            ----------
            name : {str}
                Name of the histogram/branch
            binning : {tuple, array-like}, optional
                If bining is an tuple, it defines equal-width bins in the given range. If bins is an array-like, it defines the bin edges, including the rightmost edge, allowing for non-uniform bin widths.
                (the default is (20, -1000, 1000))
            script : {str}, optional
                [description] (the default is None, which does nothing)
            style : {str}, optional
                This should be either "foreach" or "single". Decide how you fill the hist/branch (the default is 'single')
            do : {list[str]}, optional
                This should be either or both "hist" and "tree"
            only : {list[str]}, optional
                This should be either or both "be", "bmu", "re" or "rmu" (the default is None, which means fill it for all channels)
            title : {str}, optional
                Title of the histogram (the default is '{self.name}')
            """
            self.name = name
            self.binning = binning
            self.title = title.format(self = self)
            self.script = compile(script, '<TopNtupleAnalysis.Observable>', 'eval')
            self.style = style
            self.only = only
            self.do = do
            self.dtype = dtype
            self.default = default
            self.need_truth = need_truth
    def registered(self, analysis):
        return  _Observable(self.name, analysis, binning = self.binning, script = self.script, style = self.style, only = self.only, title = self.title, do = self.do, dtype = self.dtype, default = self.default, need_truth = self.need_truth)
    def queue(self):
        ObservableList.append(self)

class _Observable(object):
    def __init__(self, name, analysis, binning = (20, -1000, 1000), script = None, style = 'single', do = ['hist'], only = None, title = '{self.name}', dtype = None, default = -1, need_truth = False):
        self.name = name
        self.binning = binning
        self.title = title.format(self = self)
        self.script = script
        self.style = style
        self.only = only
        self.analysis = analysis
        self.do = do
        self.dtype = dtype
        self.default = default
        self.need_truth = need_truth
        self._globals = {'analysis': self.analysis}
        self._globals.update(self.analysis.run.im_func.func_globals)
        self._globals.update(globals())
    def __call__(self, _type = None, _locals = None):
        if self.script != None:
            if _locals != None:
                self._globals.update(_locals)
            ret = eval(self.script, self._globals)
            if _type != None:
                return _type(ret)
            else:
                return ret

### Chi2 ###
# Observable("chi2", do = ['tree'], only = ['re','rmu'],   script = """analysis.TtresChi2.chi2""").queue()
# Observable("chi2_mtl", (150, 0, 300), do = ['tree','hist'], only = ['re','rmu'],   script = """analysis.TtresChi2.mtl""").queue()
# Observable("chi2_mth", (150, 0, 300), do = ['tree','hist'], only = ['re','rmu'],   script = """analysis.TtresChi2.mth""").queue()
# Observable("chi2_mwh", (150, 0, 300), do = ['tree','hist'], only = ['re','rmu'],   script = """analysis.TtresChi2.mwh""").queue()

# Observable("chi2_Th", do = ['tree'], only = ['re','rmu'], dtype = 'TLorentzVector', style = 'foreach', script = """[analysis.TtresChi2.get_tv("Th")]""").queue()
# Observable("chi2_Wh", do = ['tree'], only = ['re','rmu'], dtype = 'TLorentzVector', style = 'foreach', script = """[analysis.TtresChi2.get_tv("Wh")]""").queue()
# Observable("chi2_Tl", do = ['tree'], only = ['re','rmu'], dtype = 'TLorentzVector', style = 'foreach', script = """[analysis.TtresChi2.get_tv("Tl")]""").queue()
# Observable("chi2_Wl", do = ['tree'], only = ['re','rmu'], dtype = 'TLorentzVector', style = 'foreach', script = """[analysis.TtresChi2.get_tv("Wl")]""").queue()
### Track Jets ###
# Observable("trackJetEta", (50, -4, 4), """(tjet.Eta() for tjet in analysis.bot_tagger._tjet_p4)""", style = 'foreach').queue()
# Observable("trackJetPt", (50, 0, 500), """(tjet.Pt()*1e-3 for tjet in analysis.bot_tagger._tjet_p4)""", style = 'foreach', do = ['tree', 'hist']).queue()
# Observable("trackJetIsBtagged", dtype = int, do = ['tree'], script = '''analysis.bot_tagger.tjet_isbtagged''', style = 'foreach').queue()
# Observable("trackJetTrueFlav", dtype = int, do = ['tree'], script = '''sel.tjet_label''', style = 'foreach').queue()
# Observable("trackJetPhi", (50, -math.pi, math.pi), """(tjet.Phi() for tjet in analysis.bot_tagger._tjet_p4)""", style = 'foreach').queue()
# Observable("btaggedtrackJetEta", (50, -4, 4), do = ['tree','hist'], script = """(tjet.Eta() for i, tjet in enumerate(analysis.bot_tagger._tjet_p4) if helpers.char2int(analysis.bot_tagger.tjet_isbtagged[i]))""", style = 'foreach').queue()
# Observable("btaggedtrackJetPt", (50, 0, 500), do = ['tree','hist'], script = """(tjet.Pt()*1e-3 for i, tjet in enumerate(analysis.bot_tagger._tjet_p4) if helpers.char2int(analysis.bot_tagger.tjet_isbtagged[i]))""", style = 'foreach').queue()
# Observable("btaggedtrackJetPhi", (50, -math.pi, math.pi), do = ['tree','hist'], script = """(tjet.Phi() for i, tjet in enumerate(analysis.bot_tagger._tjet_p4) if helpers.char2int(analysis.bot_tagger.tjet_isbtagged[i]))""", style = 'foreach').queue()
# Observable("trackJetdeltaPhilep", (50, -math.pi, math.pi), """(ROOT.Math.VectorUtil.DeltaPhi(tjet, l) for tjet in analysis.bot_tagger._tjet_p4)""", style = 'foreach').queue()
# Observable("trackJetdeltaRlep", (50, 0, (math.pi**2+2.5**2)**0.5), """(ROOT.Math.VectorUtil.DeltaR(tjet, l) for tjet in analysis.bot_tagger._tjet_p4)""", style = 'foreach').queue()
# Observable("btaggedtrackJetdeltaPhilep", (50, -math.pi, math.pi), """(ROOT.Math.VectorUtil.DeltaPhi(tjet, l) for i, tjet in enumerate(analysis.bot_tagger._tjet_p4) if helpers.char2int(analysis.bot_tagger.tjet_isbtagged[i]))""", style = 'foreach').queue()
# Observable("btaggedtrackJetdeltaRlep", (50, 0, (math.pi**2+2.5**2)**0.5), """(ROOT.Math.VectorUtil.DeltaR(tjet, l) for i, tjet in enumerate(analysis.bot_tagger._tjet_p4) if helpers.char2int(analysis.bot_tagger.tjet_isbtagged[i]))""", style = 'foreach').queue()
#Observable("dNtruthMatchedBTrackJetdPt", (30, 0, 3000), do = ['hist', 'tree'], script = """(p4.Pt()*1e-3 for (p4, isbtagged, istrueb) in zip(analysis.bot_tagger._tjet_p4, analysis.bot_tagger.tjet_isbtagged, analysis.bot_tagger.tjet_istrueb) if (isbtagged and istrueb))""", style = 'foreach', need_truth = True).queue()
#Observable("dNtruedBTrackJetdPt", (30, 0, 3000), do = ['hist', 'tree'], script = """(p4.Pt()*1e-3 for (p4, istrueb) in zip(analysis.bot_tagger._tjet_p4, analysis.bot_tagger.tjet_istrueb) if istrueb)""", style = 'foreach', need_truth = True).queue()
# Observable("NtrueB", (6, 0, 6), do = ['tree','hist'], dtype = int, style = 'single', script = """sum(analysis.bot_tagger.tjet_istrueb)""", need_truth = True).queue()
# Observable("NtaggedtrueB", (6, 0, 6), do = ['tree','hist'], dtype = int, style = 'single', script = """sum(isbtagged*istrueb for isbtagged, istrueb in zip(analysis.bot_tagger.tjet_isbtagged,analysis.bot_tagger.tjet_istrueb))""", need_truth = True).queue()

# Observable("NtrueBfromT", (3, 0, 3), do = ['tree','hist'], need_truth = True, style = 'single', dtype = int,
#            script = """any(ROOT.Math.VectorUtil.DeltaR(ROOT.Math.PtEtaPhiMVector(sel.MC_b_from_t_pt, sel.MC_b_from_t_eta, sel.MC_b_from_t_phi, sel.MC_b_from_t_m), tjet_p4)<0.4 for tjet_p4, isbtagged in zip(analysis.bot_tagger._tjet_p4, analysis.bot_tagger.tjet_isbtagged) if helpers.char2int(isbtagged))""" + 
#                    """+any(ROOT.Math.VectorUtil.DeltaR(ROOT.Math.PtEtaPhiMVector(sel.MC_b_from_tbar_pt, sel.MC_b_from_tbar_eta, sel.MC_b_from_tbar_phi, sel.MC_b_from_tbar_m), tjet_p4)<0.4 for tjet_p4, isbtagged in zip(analysis.bot_tagger._tjet_p4, analysis.bot_tagger.tjet_isbtagged) if helpers.char2int(isbtagged))""").queue()
# Observable("dNtruthMatchedBTrackJetdPt_lepside", (25, 0, 2500), """(p4.Pt()*1e-3 for (p4, isbtagged, istrueb) in zip(analysis.bot_tagger._tjet_p4, analysis.bot_tagger.tjet_isbtagged, analysis.bot_tagger.tjet_istrueb) if (isbtagged and istrueb and ROOT.Math.VectorUtil.DeltaR(p4, tlep)<1.0))""", style = 'foreach').queue()
# Observable("dNtruedBTrackJetdPt_lepside", (25, 0, 2500), """(p4.Pt()*1e-3 for (p4, istrueb) in zip(analysis.bot_tagger._tjet_p4, analysis.bot_tagger.tjet_istrueb) if istrueb and ROOT.Math.VectorUtil.DeltaR(p4, tlep)<1.0)""", style = 'foreach').queue()
# Observable("dNtruthMatchedBTrackJetdPt_hadside", (25, 0, 2500), """(p4.Pt()*1e-3 for (p4, isbtagged, istrueb) in zip(analysis.bot_tagger._tjet_p4, analysis.bot_tagger.tjet_isbtagged, analysis.bot_tagger.tjet_istrueb) if (isbtagged and istrueb and ROOT.Math.VectorUtil.DeltaR(p4, lj)<1.0))""", style = 'foreach').queue()
# Observable("dNtruedBTrackJetdPt_hadside", (25, 0, 2500), """(p4.Pt()*1e-3 for (p4, istrueb) in zip(analysis.bot_tagger._tjet_p4, analysis.bot_tagger.tjet_istrueb) if istrueb and ROOT.Math.VectorUtil.DeltaR(p4, lj)<1.0)""", style = 'foreach').queue()
# Observable("NB_lepside", (4, 0, 4), """sel.NB_lepside""").queue()
# Observable("NB_hadside", (4, 0, 4), """sel.NB_hadside""").queue()
# Observable("MC_b_from_t", do = ['tree'], only = ['bFH'], style = 'foreach', dtype = 'ROOT::Math::PtEtaPhiMVector', need_truth = True,
#            script = """[(sel.MC_b_from_t_pt, sel.MC_b_from_t_eta, sel.MC_b_from_t_phi, sel.MC_b_from_t_m)]""").queue()
# Observable("MC_b_from_tbar", do = ['tree'], only = ['bFH'], style = 'foreach', dtype = 'ROOT::Math::PtEtaPhiMVector', need_truth = True,
#            script = """[(sel.MC_b_from_tbar_pt, sel.MC_b_from_tbar_eta, sel.MC_b_from_tbar_phi, sel.MC_b_from_tbar_m)]""").queue()

### Basic Object ###
Observable("met_met", do = ['tree'],   script = """sel.met_met*1e-3""").queue()
Observable("met_phi", do = ['tree'],   script = """sel.met_phi""").queue()
Observable("jet_N", do = ['tree'], style = 'single', dtype = int,  script = """sel.jet_pt.size()""").queue()
Observable("ljet_N", do = ['tree'], style = 'single', dtype = int,  script = """sel.ljet_pt.size()""").queue()
Observable("tjet_N", do = ['tree'], style = 'single', dtype = int,  script = """sel.tjet_pt.size()""").queue()
Observable("btjet_N", do = ['tree'], style = 'single', dtype = int,  script = """sum(helpers.char2int(tjet_isbtagged) for tjet_isbtagged in analysis.bot_tagger.tjet_isbtagged)""").queue()
#######
## jets
Observable("jet_pt", do = ['tree'], style = 'foreach', script = """[jpt*1e-3 for jpt in sel.jet_pt]""").queue()
Observable("jet_eta", do = ['tree'], style = 'foreach', script = """[jeta for jeta in sel.jet_eta]""").queue()
Observable("jet_phi", do = ['tree'], style = 'foreach', script = """[jphi for jphi in sel.jet_phi]""").queue()
Observable("jet_e", do = ['tree'], style = 'foreach', script = """[je*1e-3 for je in sel.jet_e]""").queue()
Observable("jet_isbtagged", do = ['tree'], style = 'foreach', dtype = bool, script = """[bool(btag) for btag in self.bot_tagger.jet_isbtagged]""").queue()

# closest jet to lepton that is used to reconstruct leptonic top
Observable("closestjet_idx", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = int, script = """closeJetIdx""")
#
Observable("closestjet_pt", do = ['tree'], only = ['be', 'bmu'], script = """closeJet.Pt()*1e-3""").queue()
Observable("closestjet_eta", do = ['tree'], only = ['be', 'bmu'], script = """closeJet.Eta()""").queue()
Observable("closestjet_phi", do = ['tree'], only = ['be', 'bmu'], script = """closeJet.Phi()""").queue()
Observable("closestjet_e", do = ['tree'], only = ['be', 'bmu'], script = """closeJet.E()*1e-3""").queue()

## lepton
Observable("lep_pt", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = float, script = """l.Pt()*1e-3""").queue()
Observable("lep_eta", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = float, script = """l.Eta()""").queue()
Observable("lep_phi", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = float, script = """l.Phi()""").queue()
Observable("lep_e", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = float, script = """l.E()*1e-3""").queue()
Observable("lep_charge", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = int, script = """int(lQ)""").queue()

## kinematic fit
## neutrino
Observable("nu_pt", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """nu.Pt()*1e-3""").queue()
Observable("nu_eta", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """nu.Eta()""").queue()
Observable("nu_phi", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """nu.Phi()""").queue()

# reco leptonic top
# boosted case
Observable("tl_pt", do = ['tree'], only = ['be', 'bmu'], script = """tlep.Pt()*1e-3""").queue()
Observable("tl_eta", do = ['tree'], only = ['be', 'bmu'], script = """tlep.Eta()""").queue()
Observable("tl_phi", do = ['tree'], only = ['be', 'bmu'], script = """tlep.Phi()""").queue()
Observable("tl_m", do = ['tree'], only = ['be', 'bmu'], script = """tlep.M()*1e-3""").queue()
#resolved case
Observable("tl_pt", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Tl").Pt()""").queue()
Observable("tl_eta", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Tl").Eta()""").queue()
Observable("tl_phi", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Tl").Phi()""").queue()
Observable("tl_m", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Tl").M()""").queue()

# reco hadronic top
# boosted case
Observable("th_pt", do = ['tree'], only = ['be', 'bmu'], script = """lj.Pt()*1e-3""").queue()
Observable("th_eta", do = ['tree'], only = ['be', 'bmu'], script = """lj.Eta()""").queue()
Observable("th_phi", do = ['tree'], only = ['be', 'bmu'], script = """lj.Phi()""").queue()
Observable("th_m", do = ['tree'], only = ['be', 'bmu'], script = """lj.M()*1e-3""").queue()
# resolved case
Observable("th_pt", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Th").Pt()""").queue()
Observable("th_eta", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Th").Eta()""").queue()
Observable("th_phi", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Th").Phi()""").queue()
Observable("th_m", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Th").M()""").queue()

# reconstred W
Observable("Wl_pt", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Wl").Pt()""").queue()
Observable("Wl_eta", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Wl").Eta()""").queue()
Observable("Wl_phi", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Wl").Phi()""").queue()
Observable("Wl_m", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Wl").M()""").queue()
Observable("Wh_pt", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Wh").Pt()""").queue()
Observable("Wh_eta", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Wh").Eta()""").queue()
Observable("Wh_phi", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Wh").Phi()""").queue()
Observable("Wh_m", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Wh").M()""").queue()

# MC truth
# hadronic top
Observable("th_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """th_mc.Pt()""", need_truth = True).queue()
Observable("th_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """th_mc.Eta()""", need_truth = True).queue()
Observable("th_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """th_mc.Phi()""", need_truth = True).queue()
Observable("th_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """th_mc.M()""", need_truth = True).queue()
# leptonic top
Observable("tl_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """tl_mc.Pt()""", need_truth = True).queue()
Observable("tl_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """tl_mc.Eta()""", need_truth = True).queue()
Observable("tl_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """tl_mc.Phi()""", need_truth = True).queue()
Observable("tl_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """tl_mc.M()""", need_truth = True).queue()

# b from hadronic top
Observable("bh_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_bh_pt*1e-3""", need_truth = True).queue()
Observable("bh_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_bh_eta""", need_truth = True).queue()
Observable("bh_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_bh_phi""", need_truth = True).queue()
Observable("bh_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_bh_m*1e-3""", need_truth = True).queue()
# b from leptonic top
Observable("bl_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_bl_pt*1e-3""", need_truth = True).queue()
Observable("bl_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_bl_eta""", need_truth = True).queue()
Observable("bl_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_bl_phi""", need_truth = True).queue()
Observable("bl_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_bl_m*1e-3""", need_truth = True).queue()

# hadronic W
Observable("Wh_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_wh_pt*1e-3""", need_truth = True).queue()
Observable("Wh_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_wh_eta""", need_truth = True).queue()
Observable("Wh_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_wh_phi""", need_truth = True).queue()
Observable("Wh_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_wh_m*1e-3""", need_truth = True).queue()
#leptonic W
Observable("Wl_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_wl_pt*1e-3""", need_truth = True).queue()
Observable("Wl_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_wl_eta""", need_truth = True).queue()
Observable("Wl_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_wl_phi""", need_truth = True).queue()
Observable("Wl_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_wl_m*1e-3""", need_truth = True).queue()

# hadronic W daughters
Observable("WhDecay1_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1h_pt*1e-3""", need_truth = True).queue()
Observable("WhDecay1_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1h_eta""", need_truth = True).queue()
Observable("WhDecay1_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1h_phi""", need_truth = True).queue()
Observable("WhDecay1_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1h_m*1e-3""", need_truth = True).queue()
Observable("WhDecay1_pdgid_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = int, script = """sel.MC_w1h_pdgId""", need_truth = True).queue()
Observable("WhDecay2_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w2h_pt*1e-3""", need_truth = True).queue()
Observable("WhDecay2_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w2h_eta""", need_truth = True).queue()
Observable("WhDecay2_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w2h_phi""", need_truth = True).queue()
Observable("WhDecay2_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w2h_m*1e-3""", need_truth = True).queue()
Observable("WhDecay2_pdgid_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = int, script = """sel.MC_w2h_pdgId""", need_truth = True).queue()

# leptonic W daughters
Observable("WlDecay1_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_pt*1e-3""", need_truth = True).queue()
Observable("WlDecay1_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_eta""", need_truth = True).queue()
Observable("WlDecay1_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_phi""", need_truth = True).queue()
Observable("WlDecay1_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_m*1e-3""", need_truth = True).queue()
Observable("WlDecay1_pdgid_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = int, script = """sel.MC_w1l_pdgId""", need_truth = True).queue()
Observable("WlDecay2_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w2l_pt*1e-3""", need_truth = True).queue()
Observable("WlDecay2_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w2l_eta""", need_truth = True).queue()
Observable("WlDecay2_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w2l_phi""", need_truth = True).queue()
Observable("WlDecay2_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w2l_m*1e-3""", need_truth = True).queue()
Observable("WlDecay2_pdgid_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], dtype = int, script = """sel.MC_w2l_pdgId""", need_truth = True).queue()

# truth lepton
Observable("lep_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_pt*1e-3 if sel.MC_w1l_pdgId%2 else sel.MC_w2l_pt*1e-3""", need_truth = True).queue()
Observable("lep_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_eta if sel.MC_w1l_pdgId%2 else sel.MC_w2l_eta""", need_truth = True).queue()
Observable("lep_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_phi if sel.MC_w1l_pdgId%2 else sel.MC_w2l_phi""", need_truth = True).queue()
Observable("lep_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_m*1e-3 if sel.MC_w1l_pdgId%2 else sel.MC_w2l_m*1e-3""", need_truth = True).queue()
Observable("nu_pt_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_pt*1e-3 if not sel.MC_w1l_pdgId%2 else sel.MC_w2l_pt*1e-3""", need_truth = True).queue()
Observable("nu_eta_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_eta if not sel.MC_w1l_pdgId%2 else sel.MC_w2l_eta""", need_truth = True).queue()
Observable("nu_phi_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_phi if not sel.MC_w1l_pdgId%2 else sel.MC_w2l_phi""", need_truth = True).queue()
Observable("nu_m_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """sel.MC_w1l_m*1e-3 if not sel.MC_w1l_pdgId%2 else sel.MC_w2l_m*1e-3""", need_truth = True).queue()

Observable("th1_pt", do = ['tree'], only = ['bFH'],  script = """lj1.Pt()""").queue()
Observable("th1_eta", do = ['tree'], only = ['bFH'], script = """lj1.Eta()""").queue()
Observable("th1_phi", do = ['tree'], only = ['bFH'], script = """lj1.Phi()""").queue()
Observable("th1_m", do = ['tree'], only = ['bFH'],   script = """lj1.M()""").queue()
Observable("th1_DNN", do = ['tree'], only = ['bFH'], script = """sel.ljet_DNNContainedTopTag_score[goodJetIdx1] if goodJetIdx1 != -1 else -999""").queue()
Observable("th1_SF", do = ['tree'], only = ['bFH'], script = """analysis.top_tagger.ljet_toptagSF[goodJetIdx1]""", need_truth = True).queue()
Observable("th1_isleading", do = ['tree'], only = ['bFH'], dtype = int,  script = """goodJetIdx1==0""").queue()
Observable("th2_pt", do = ['tree'], only = ['bFH'],  script = """lj2.Pt()""").queue()
Observable("th2_eta", do = ['tree'], only = ['bFH'], script = """lj2.Eta()""").queue()
Observable("th2_phi", do = ['tree'], only = ['bFH'], script = """lj2.Phi()""").queue()
Observable("th2_m", do = ['tree'], only = ['bFH'],   script = """lj2.M()""").queue()
Observable("th2_DNN", do = ['tree'], only = ['bFH'],   script = """sel.ljet_DNNContainedTopTag_score[goodJetIdx2] if goodJetIdx2 != -1 else -999""").queue()
Observable("th2_SF", do = ['tree'], only = ['bFH'], script = """analysis.top_tagger.ljet_toptagSF[goodJetIdx2]""", need_truth = True).queue()
Observable("th2_issubleading", do = ['tree'], only = ['bFH'], dtype = int,  script = """goodJetIdx2==1""").queue()

Observable("toptagged_N", (4, 0, 4), do = ['hist', 'tree'], only = ['bFH'], style = 'single', dtype = int,  script = """sum(analysis.top_tagger.ljet_istoptagged)""").queue()

Observable("truth_ljet_pt_MA",  do = ['tree'], style = 'foreach', dtype = float, script = """(analysis.top_tagger.truth_ljet_p4[i].Pt()*1e-3 for i in analysis.top_tagger.ljet_truthjetid if i >= 0)""", need_truth = True).queue()
Observable("truth_ljet_eta_MA", do = ['tree'], style = 'foreach', dtype = float, script = """(analysis.top_tagger.truth_ljet_p4[i].Eta() for i in analysis.top_tagger.ljet_truthjetid if i >= 0)""",     need_truth = True).queue()
Observable("truth_ljet_phi_MA", do = ['tree'], style = 'foreach', dtype = float, script = """(analysis.top_tagger.truth_ljet_p4[i].Phi() for i in analysis.top_tagger.ljet_truthjetid if i >= 0)""",     need_truth = True).queue()
Observable("truth_ljet_m_MA"  , do = ['tree'], style = 'foreach', dtype = float, script = """(analysis.top_tagger.truth_ljet_p4[i].M()*1e-3 for i in analysis.top_tagger.ljet_truthjetid if i >= 0)""",  need_truth = True).queue()
#Observable("truth", do = ['tree'], need_truth = True, style = 'foreach', dtype = 'ROOT::Math::PtEtaPhiMVector', script = """[ROOT.Math.PtEtaPhiMVector(sel.truthparticle_pt.at(i), sel.truthparticle_eta.at(i), sel.truthparticle_phi.at(i), sel.truthparticle_m.at(i))*1e-3 for i in xrange(sel.truthparticle_pt.size())]""").queue()
#Observable("truth_id", do = ['tree'], need_truth = True, style = 'foreach', dtype = int, script = """sel.truthparticle_type""").queue()
#Observable("akt10truthjet", do = ['tree'], only = ['bFH'], style = 'foreach', dtype = 'TLorentzVector', need_truth = True,
#           script = """[P4fromPtEtaPhiM(sel.akt10truthjet_pt[i], sel.akt10truthjet_eta[i], sel.akt10truthjet_phi[i], sel.akt10truthjet_m[i]) for i in xrange(sel.akt10truthjet_pt.size())]""").queue()


# Observable('trkbjets_N', (4, 0, 5), do = ['hist', 'tree'], dtype = int, script = """sum(map(helpers.char2int, analysis.bot_tagger.tjet_isbtagged))""").queue()

### Jet Substructure ###
Observable("th_tau21_wta", do = ['tree'], only = ['be', 'bmu'], script = """sel.ljet_tau21_wta[goodJetIdx] if goodJetIdx != -1 else -999""").queue()
Observable("th_tau32_wta", do = ['tree'], only = ['be', 'bmu'], script = """sel.ljet_tau32_wta[goodJetIdx] if goodJetIdx != -1 else -999""").queue()
Observable("th1_tau21_wta", do = ['tree'], only = ['bFH'], script = """sel.ljet_tau21_wta[goodJetIdx1] if goodJetIdx1 != -1 else -999""").queue()
Observable("th1_tau32_wta", do = ['tree'], only = ['bFH'], script = """sel.ljet_tau32_wta[goodJetIdx1] if goodJetIdx1 != -1 else -999""").queue()
Observable("th2_tau21_wta", do = ['tree'], only = ['bFH'], script = """sel.ljet_tau21_wta[goodJetIdx2] if goodJetIdx2 != -1 else -999""").queue()
Observable("th2_tau32_wta", do = ['tree'], only = ['bFH'], script = """sel.ljet_tau32_wta[goodJetIdx2] if goodJetIdx2 != -1 else -999""").queue()
# Observable("th_C2", (26, 0, 2), do = ['hist','tree'], only = ['b'], script = """sel.ljet_C2[analysis.top_tagger.thad_index] if analysis.top_tagger.thad_index != -1 else -999""").queue()
# Observable("th_D2", (26, 0, 5), do = ['hist','tree'], only = ['b'], script = """sel.ljet_D2[analysis.top_tagger.thad_index] if analysis.top_tagger.thad_index != -1 else -999""").queue()
# Observable("th_MClike", (4, -1, 3), dtype = int, do = ['hist','tree'], only = ['b'], script = """sel.ljet_MClike[analysis.top_tagger.thad_index] if analysis.top_tagger.thad_index != -1 else -1""").queue()

### Pileup ###
Observable("mu", do = ['tree'], style = 'single', script = """sel.mu""").queue()
Observable("npv", do = ['tree'], style = 'single', dtype = int, script = """sel.npv""").queue()
Observable("vtxz", do = ['tree'], style = 'single', script = """sel.vtxz""").queue()

### ttbar kinematics ###
# reco level
# boosted case
Observable("pttReco", do = ['tree'], only = ['be', 'bmu'], script = """(tlep+lj).Pt()*1e-3""").queue()
Observable("yttReco", do = ['tree'], only = ['be', 'bmu'], script = """(tlep+lj).Rapidity()""").queue()
Observable("ystarReco", do = ['tree'], only = ['be', 'bmu'], script = """abs(tlep.Rapidity()-lj.Rapidity())/2""").queue()
Observable("dphiReco", do = ['tree'], only = ['be', 'bmu'], script = """abs(ROOT.TVector2.Phi_mpi_pi(tlep.Phi()-lj.Phi()))""").queue()
Observable("HtReco", do = ['tree'], only = ['be', 'bmu'], script = """(tlep.Pt()+lj.Pt())*1e-3""").queue()
Observable("yboostReco", do = ['tree'], only = ['be', 'bmu'], script = """(tlep.Rapidity()+lj.Rapidity())/2""").queue()
Observable("th_y", do = ['tree'], only = ['be', 'bmu'], script = """lj.Rapidity()""").queue()
Observable("tl_y", do = ['tree'], only = ['be', 'bmu'], script = """tlep.Rapidity()""").queue()
Observable("th_pout", do = ['tree'], only = ['be', 'bmu'], script = """tlep.Vect().Unit().Cross(ROOT.TVector3(0,0,1)).Dot(lj.Vect())*1e-3""").queue()
Observable("tl_pout", do = ['tree'], only = ['be', 'bmu'], script = """lj.Vect().Unit().Cross(ROOT.TVector3(0,0,1)).Dot(tlep.Vect())*1e-3""").queue()
# resolved case
Observable("pttReco", do = ['tree'], only = ['re', 'rmu'], script = """(self.TtresChi2.get_tv("Th")+self.TtresChi2.get_tv("Tl")).Pt()""").queue()
Observable("yttReco", do = ['tree'], only = ['re', 'rmu'], script = """(self.TtresChi2.get_tv("Th")+self.TtresChi2.get_tv("Tl")).Rapidity()""").queue()
Observable("ystarReco", do = ['tree'], only = ['re', 'rmu'], script = """abs(self.TtresChi2.get_tv("Tl").Rapidity()-self.TtresChi2.get_tv("Th").Rapidity())/2""").queue()
Observable("dphiReco", do = ['tree'], only = ['re', 'rmu'], script = """abs(ROOT.TVector2.Phi_mpi_pi(self.TtresChi2.get_tv("Tl").Phi()-self.TtresChi2.get_tv("Th").Phi()))""").queue()
Observable("HtReco", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Tl").Pt()+self.TtresChi2.get_tv("Th").Pt()""").queue()
Observable("yboostReco", do = ['tree'], only = ['re', 'rmu'], script = """(self.TtresChi2.get_tv("Tl").Rapidity()+self.TtresChi2.get_tv("Th").Rapidity())/2""").queue()
Observable("th_y", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Th").Rapidity()""").queue()
Observable("tl_y", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Tl").Rapidity()""").queue()
Observable("th_pout", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Tl").Vect().Unit().Cross(ROOT.TVector3(0,0,1)).Dot(self.TtresChi2.get_tv("Th").Vect())""").queue()
Observable("tl_pout", do = ['tree'], only = ['re', 'rmu'], script = """self.TtresChi2.get_tv("Th").Vect().Unit().Cross(ROOT.TVector3(0,0,1)).Dot(self.TtresChi2.get_tv("Tl").Vect())""").queue()

# truth level
Observable("pttTrue", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """ttbar_mc.Pt()""", need_truth = True).queue()
Observable("yttTrue", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """ttbar_mc.Rapidity()""", need_truth = True).queue()
Observable("ystarTrue", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """abs(th_mc.Rapidity()-tl_mc.Rapidity())/2""", need_truth = True).queue()
Observable("dphiTrue", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """abs(ROOT.TVector2.Phi_mpi_pi(th_mc.Phi()-tl_mc.Phi()))""", need_truth = True).queue()
Observable("HtTrue", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """th_mc.Pt()+tl_mc.Pt()""", need_truth = True).queue()
Observable("yboostTrue", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """(th_mc.Rapidity()+tl_mc.Rapidity())/2""", need_truth = True).queue()
Observable("th_y_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """th_mc.Rapidity()""", need_truth = True).queue()
Observable("tl_y_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """tl_mc.Rapidity()""", need_truth = True).queue()
Observable("th_pout_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """tl_mc.Vect().Unit().Cross(ROOT.TVector3(0,0,1)).Dot(th_mc.Vect())""", need_truth = True).queue()
Observable("tl_pout_MC", do = ['tree'], only = ['re', 'rmu', 'be', 'bmu'], script = """th_mc.Vect().Unit().Cross(ROOT.TVector3(0,0,1)).Dot(tl_mc.Vect())""", need_truth = True).queue()


ModuleNotFoundError: No module named 'ROOT'